In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder

In [4]:
df = pd.read_csv("/Users/deepthiramesh/Documents/Extra Projects/Supply chain delay analysis/Dataset/dynamic_supply_chain_logistics_dataset_with_country.csv")

In [6]:
# Step 3: Data Cleaning & Parsing with synthetic delivery dates
num_rows = len(df)
base_dates = pd.date_range(start='2011-01-01', periods=1825, freq='D')
repeated_dates = np.tile(base_dates, int(np.ceil(num_rows / len(base_dates))))[:num_rows]

df['Scheduled Delivery Date'] = repeated_dates
df['Delivered to Client Date'] = df['Scheduled Delivery Date'] + pd.to_timedelta(df['delivery_time_deviation'].fillna(0), unit='D')
df['delay_days'] = (df['Delivered to Client Date'] - df['Scheduled Delivery Date']).dt.days
df = df[df['delay_days'].notnull()]

In [7]:
# Step 4: Delay Analysis
df['Route'] = df['supplier_country'] + " → Destination"
df['Delivery Hour'] = df['Delivered to Client Date'].dt.hour
df['Delivery Day'] = df['Delivered to Client Date'].dt.day_name()

In [8]:
# Step 5: Delay Summary
delay_summary = df.groupby('Route')['delay_days'].agg(['mean', 'std', 'count']).reset_index()
carrier_delay = df.groupby('supplier_id')['delay_days'].mean().sort_values(ascending=False)